# Memory Grant Feedback
Memory grant feedback (MGF)adjusts memory grant sizes to learn and improve memory usage. This feature can adjust memory grant sizes for both batch and row mode operators, and eliminate the effects of memory grant misestimations (spills to TempDB) and overestimations (affects concurrency).

Memory grant feedback is a feature under the [**Intelligent Query Processing**](https://aka.ms/iqp) suite of features.  

This example will show you how upgrading to **Database Compatibility Level 150** could improve performance of queries executing in row mode, that are affected by memory grant misestimations. Upgrading to **Database Compatibility Level 140** allows MGF for queries executing in batch mode, and **Database Compatibility Level 150** allows MGF on queries executing in row mode.

More information about this feature is available [here](https://docs.microsoft.com/sql/relational-databases/performance/intelligent-query-processing?view=sql-server-ver15#batch-mode-memory-grant-feedback).

## Step 1: Setup WideWorldImportersDW database

You could choose to use a container to evaluate this feature. Create an instance of SQL Server 2019 using a Docker image and restore the WideWorldImportersDW database backup

You will need the **WideWorldImportersDW** database for this exercise. If you don't have this sample database, then you download the sample database [here](https://github.com/Microsoft/sql-server-samples/releases/download/wide-world-importers-v1.0/WideWorldImportersDW-Full.bak "WideWorldImportersDW-Full download").

Restore the copied WideWorldImportersDW database backup into the container and restore the backup.

##### Docker Commands
```
docker pull mcr.microsoft.com/mssql/server:2019-latest

docker run -e "ACCEPT_EULA=Y" -e "SA_PASSWORD=`<A Strong Password`>" -p 1445:1433 --name sql2019demo -d mcr.microsoft.com/mssql/server:2019-latest

docker cp ".\Downloads\WideWorldImportersDW-Full.bak" sql2019demo:/var/opt/mssql/data
```

**Note**: *For Linux installations the default path to use is /var/opt/mssql*


In [7]:
USE [master]
GO
IF EXISTS (SELECT [database_id] FROM sys.databases WHERE [name] = 'WideWorldImportersDW')
ALTER DATABASE [WideWorldImportersDW] SET SINGLE_USER WITH ROLLBACK IMMEDIATE
GO

DECLARE @datafilepath VARCHAR(8000) = CAST(SERVERPROPERTY('InstanceDefaultDataPath') AS VARCHAR(4000)) + 'WideWorldImportersDW.mdf'
DECLARE @logfilepath VARCHAR(8000) = CAST(SERVERPROPERTY('InstanceDefaultLogPath') AS VARCHAR(4000)) + 'WideWorldImportersDW.ldf'
DECLARE @inmemfilepath VARCHAR(8000) = CAST(SERVERPROPERTY('InstanceDefaultDataPath') AS VARCHAR(4000)) + 'WideWorldImportersDW_InMemory_Data_1'
DECLARE @secondaryfilepath VARCHAR(8000) = CAST(SERVERPROPERTY('InstanceDefaultDataPath') AS VARCHAR(4000))+ 'WideWorldImportersDW_2.ndf'

-- Change @backupfile file path as needed
DECLARE @backupfile VARCHAR(8000) = 'E:\SampleDBs\WideWorldImportersDW-Full.bak'
RESTORE DATABASE WideWorldImportersDW
FROM DISK = @backupfile 
WITH MOVE 'WWI_Primary' TO @datafilepath,
    MOVE 'WWI_UserData' TO @secondaryfilepath,
    MOVE 'WWIDW_InMemory_Data_1' TO @inmemfilepath,
    MOVE 'WWI_Log' TO @logfilepath, NOUNLOAD, REPLACE, STATS = 10
GO

USE [master]
GO
ALTER DATABASE [WideWorldImportersDW] MODIFY FILE ( NAME = N'WWI_Log', SIZE = 4GB )
GO

## Step 2: Enlarge the WideWorldImportersDW database

In [3]:
USE WideWorldImportersDW;
GO

/*
Assumes a fresh restore of WideWorldImportersDW
*/

IF OBJECT_ID('Fact.OrderHistory') IS NULL 
BEGIN
    SELECT [Order Key], [City Key], [Customer Key], [Stock Item Key], [Order Date Key], [Picked Date Key], [Salesperson Key], [Picker Key], [WWI Order ID], [WWI Backorder ID], Description, Package, Quantity, [Unit Price], [Tax Rate], [Total Excluding Tax], [Tax Amount], [Total Including Tax], [Lineage Key]
    INTO Fact.OrderHistory
    FROM Fact.[Order];
END;

ALTER TABLE Fact.OrderHistory
ADD CONSTRAINT PK_Fact_OrderHistory PRIMARY KEY NONCLUSTERED ([Order Key] ASC, [Order Date Key] ASC) WITH (DATA_COMPRESSION = PAGE);
GO

CREATE INDEX IX_Stock_Item_Key
ON Fact.OrderHistory ([Stock Item Key])
INCLUDE(Quantity)
WITH (DATA_COMPRESSION = PAGE);
GO

CREATE INDEX IX_OrderHistory_Quantity
ON Fact.OrderHistory ([Quantity])
INCLUDE([Order Key])
WITH (DATA_COMPRESSION = PAGE);
GO

CREATE INDEX IX_OrderHistory_CustomerKey
ON Fact.OrderHistory([Customer Key])
INCLUDE ([Total Including Tax])
WITH (DATA_COMPRESSION = PAGE);
GO

IF (SELECT COUNT(*) FROM [Fact].[OrderHistory]) < 3702592
BEGIN
	DECLARE @i smallint
	SET @i = 0
	WHILE @i < 4
	BEGIN
		INSERT INTO [Fact].[OrderHistory] ([City Key], [Customer Key], [Stock Item Key], [Order Date Key], [Picked Date Key], [Salesperson Key], [Picker Key], [WWI Order ID], [WWI Backorder ID], Description, Package, Quantity, [Unit Price], [Tax Rate], [Total Excluding Tax], [Tax Amount], [Total Including Tax], [Lineage Key])
		SELECT [City Key], [Customer Key], [Stock Item Key], [Order Date Key], [Picked Date Key], [Salesperson Key], [Picker Key], [WWI Order ID], [WWI Backorder ID], Description, Package, Quantity, [Unit Price], [Tax Rate], [Total Excluding Tax], [Tax Amount], [Total Including Tax], [Lineage Key]
		FROM [Fact].[OrderHistory];

		SET @i = @i +1
	END;
END
GO

IF OBJECT_ID('Fact.OrderHistoryExtended') IS NULL 
BEGIN
    SELECT [Order Key], [City Key], [Customer Key], [Stock Item Key], [Order Date Key], [Picked Date Key], [Salesperson Key], [Picker Key], [WWI Order ID], [WWI Backorder ID], Description, Package, Quantity, [Unit Price], [Tax Rate], [Total Excluding Tax], [Tax Amount], [Total Including Tax], [Lineage Key]
    INTO Fact.OrderHistoryExtended
    FROM Fact.[OrderHistory];
END;

ALTER TABLE Fact.OrderHistoryExtended
ADD CONSTRAINT PK_Fact_OrderHistoryExtended PRIMARY KEY NONCLUSTERED ([Order Key] ASC, [Order Date Key] ASC)
WITH (DATA_COMPRESSION = PAGE);
GO

CREATE INDEX IX_Stock_Item_Key
ON Fact.OrderHistoryExtended ([Stock Item Key])
INCLUDE (Quantity);
GO

IF (SELECT COUNT(*) FROM [Fact].[OrderHistory]) < 29620736
BEGIN
	DECLARE @i smallint
	SET @i = 0
	WHILE @i < 3
	BEGIN
		INSERT Fact.OrderHistoryExtended([City Key], [Customer Key], [Stock Item Key], [Order Date Key], [Picked Date Key], [Salesperson Key], [Picker Key], [WWI Order ID], [WWI Backorder ID], Description, Package, Quantity, [Unit Price], [Tax Rate], [Total Excluding Tax], [Tax Amount], [Total Including Tax], [Lineage Key])
		SELECT [City Key], [Customer Key], [Stock Item Key], [Order Date Key], [Picked Date Key], [Salesperson Key], [Picker Key], [WWI Order ID], [WWI Backorder ID], Description, Package, Quantity, [Unit Price], [Tax Rate], [Total Excluding Tax], [Tax Amount], [Total Including Tax], [Lineage Key]
		FROM Fact.OrderHistoryExtended;

		SET @i = @i +1
	END;
END
GO

UPDATE Fact.OrderHistoryExtended
SET [WWI Order ID] = [Order Key];
GO

-- Repeat the following until log shrinks. These demos don't require much log space.
CHECKPOINT
GO
DBCC SHRINKFILE (N'WWI_Log' , 0, TRUNCATEONLY)
GO
SELECT * FROM sys.dm_db_log_space_usage
GO

## Step 3: Skew cardinality estimations

This will provide skewed statistical information to the Cardinality Estimator (CE). 

Statistics are fundamental building blocks for the CE to output good estimations over the expected number of rows will be used from a table for a given query.
Therefore, it's fundamental for the CE to have statistics that accurately portray the underlying data distribution of a column or set of columns. If those statistics are skewed, then cardinality estimations will likely be wrong and the Query Optimizer will likely make wrong decisions. The result is an inneficient plan for a given query.

In [1]:
USE [WideWorldImportersDW];
GO
UPDATE STATISTICS Fact.OrderHistory 
WITH ROWCOUNT = 1;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.124

## Step 4: Execute the query and observe the query execution plan


In [9]:
USE [WideWorldImportersDW];
GO
ALTER DATABASE [WideWorldImportersDW] SET COMPATIBILITY_LEVEL = 150;
GO
ALTER DATABASE SCOPED CONFIGURATION CLEAR PROCEDURE_CACHE;
GO

SELECT   
	fo.[Order Key], fo.Description,
	si.[Lead Time Days]
FROM Fact.OrderHistory AS fo
INNER HASH JOIN Dimension.[Stock Item] AS si 
	ON fo.[Stock Item Key] = si.[Stock Item Key]
WHERE fo.[Lineage Key] = 9
	AND si.[Lead Time Days] > 19;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Warning: The join order has been enforced because a local join hint is used.

(66416 rows affected)

Displaying Top 500 rows.

Total execution time: 00:00:43.190

Order Key,Description,Lead Time Days
3699679,Tape dispenser (Black),20
3700053,Tape dispenser (Black),20
3700057,Tape dispenser (Black),20
3700061,Tape dispenser (Black),20
3701657,Tape dispenser (Black),20
3701914,Tape dispenser (Black),20
3702301,Tape dispenser (Black),20
3592539,Tape dispenser (Black),20
3592540,Tape dispenser (Black),20
3592541,Tape dispenser (Black),20


Observe the query execution plan (or actual plan).

![MGF_Plan](./media/MGF_plan.PNG)

Notice the yellow triangle on the *Hash Match* operator, signaling a warning. 

Hovering over the operator brings up additional information. We see the warning detail:

*\"Operator used tempdb to spill data during execution with spill level 1 and 1 spilled thread(s), Hash wrote 52000 pages to and read 52000 pages from tempdb with granted memory 1024KB and used memory 968KB"*

![MGF_Plan_properties](./media/MGF_plan_properties.PNG)

This is a sizable spill that translates into I/O, which in most systems means slow performance.

Now click on the *SELECT* node, and look at properties of the entire plan. Specifically in the *MemoryGrantInfo* properties, note the *GrantedMemory* is 1056 KB (about 1 MB) and *IsMemoryGrantFeedbackAdjusted* has the "NoFirstExecution" state. 

![MGF_Plan_properties](./media/MGF_plan_properties_grant.PNG)

Because this was the first execution, the MGF feature didn't have a chance to learn before it executed. However, note what happens when the query is executed a few more times.

## Step 5: Execute the query again

Run the same query from Step 3 and then observe the query plan's *MemoryGrantInfo* properties. 

In [10]:
USE [WideWorldImportersDW]
GO

SELECT   
	fo.[Order Key], fo.Description,
	si.[Lead Time Days]
FROM Fact.OrderHistory AS fo
INNER HASH JOIN Dimension.[Stock Item] AS si 
	ON fo.[Stock Item Key] = si.[Stock Item Key]
WHERE fo.[Lineage Key] = 9
	AND si.[Lead Time Days] > 19;
GO

Commands completed successfully.

(66416 rows affected)

Displaying Top 500 rows.

Total execution time: 00:00:05.131

Order Key,Description,Lead Time Days
231599,Air cushion machine (Blue),20
232215,Air cushion machine (Blue),20
232290,Air cushion machine (Blue),20
231769,Air cushion machine (Blue),20
232997,Air cushion machine (Blue),20
234019,Air cushion machine (Blue),20
3702343,Air cushion machine (Blue),20
3702543,Air cushion machine (Blue),20
3700753,Air cushion machine (Blue),20
3700850,Air cushion machine (Blue),20


Observe the query execution plan (or actual plan) of the 2nd execution.

![MGF_Plan](./media/MGF_plan2.PNG)

Notice the yellow triangle on the *Hash Match* operator is gone, which means the query didn't spill and didn't incur in expensive I/O. That difference is also noticeable in the execution time. From the 1st to the 2nd execution, elapsed time dropped from **~43s** to **~5s**.

Now click on the *SELECT* node, and look at the *MemoryGrantInfo* properties. Note the *GrantedMemory* is now 625 MB (from 1056 KB) and *IsMemoryGrantFeedbackAdjusted* shows **YesAdjusting****. The MGF feature is already learning and adjusting.

On the 3rd execution, the *GrantedMemory* is still 625 MB, and *IsMemoryGrantFeedbackAdjusted* shows **YesStable**. This means the MGF feature found the optimal memory grant that's required to execute the query entirelly in memory.

### 2nd Execution
![2nd_Exec_MGF_Plan_properties](./media/MGF_plan_properties_grant2.PNG)

### 3rd Execution
![3rd_Exec_MGF_Plan_properties](./media/MGF_plan_properties_grant3.PNG)


## Step 6: Reset the skewed statistics

In [1]:
USE [WideWorldImportersDW]
GO

UPDATE STATISTICS Fact.OrderHistory 
WITH ROWCOUNT = 3702672;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.001